In [1]:
import numpy as np 
import cv2
from skimage.feature import local_binary_pattern

In [2]:
import os
directory_list = list()
dir_path = "dataset"
for root, dirs, files in os.walk(dir_path, topdown=False):
    for name in dirs:
        directory_list.append((name))
#print(directory_list)
for directory in directory_list:
    print(directory)

catagory_0
catagory_1
catagory_2
catagory_3
catagory_4
catagory_5
catagory_6
catagory_7
catagory_8
catagory_9


In [3]:
import glob

labels = 'labels.csv'
labelfile = open(labels,'w')
for i in range(len(directory_list)):
    readpath = (dir_path+'/' + directory_list[i]+'/*jpg')
    #print(readpath)
    images = glob.glob(readpath)
    for image in images:
        labelfile.write(image+','+directory_list[i][9:len(directory_list[i])]+'\n')
labelfile.close()

In [4]:
import random
labels = 'labels.csv'
shuffled_labels = 'Shuffled_labels.csv'

labelfile = open(labels, "r")
lines = labelfile.readlines()
labelfile.close()
random.shuffle(lines)

shufflefile = open(shuffled_labels, "w")
shufflefile.writelines(lines)
shufflefile.close()

In [5]:
#to check uniform or not 
def uniform(pattern):
    pat= int(pattern)
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pattern/2**(7-i))
        else:
            b= int(pattern/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pattern=pattern%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
items = []
true_items = []
for i in range(0, 256):
    if uniform(i):
        items.append(i)
        true_items.append(i)
        
items.append(250)
true_items.append(250)
print(len(items))


59


In [6]:
import cv2
import numpy as np
def get(image, idx, idy):
    if idx < (len(image)) and idy < len(image[0]) and idx>=0 and idy >=0:
        return 1
    else:
        return 0
    
def dir(x,y,img):
	ax=int(img[x+1,y])-int(img[x,y])
	ay=int(img[x,y-1])-int(img[x,y])
    
	if ax>=0 and ay>=0:
		return 1
	elif ax<0 and ay>=0:
		return 2
	elif ax<0 and ay<0:
		return 3
	elif ax>=0 and ay<0:
		return 4
		
def d_lbp(x,y,img):
    t = 0
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    val = int(img[x][y])
    pixels = []
    for i in range(0, 8):
        n_x = x+fx[i]
        n_y = y+fy[i]
        val1=int(img[n_x][n_y])
        pixels.append(val1-val)
        
    for i in range(len(pixels)):
        if pixels[i] >= pixels[(i+1)%len(pixels)] + t:
            out.append(1)
        else:
            out.append(2)
    return out

def add_dict(dic, val):
    if val not in dic.keys():
        dic[250] = dic[250] +1
    else:
        dic[val] = dic[val] + 1

def double_lbp(img, dic1, dic2):
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    for x in range(1, len(img)-1):
        for y in range(1, len(img[x])-1):
            value = d_lbp(x, y, img)
            val1=0
            val2=0
            for i in range(len(value)):
                if(value[i]==1):
                    val1+=2**(7-i)
                elif(value[i]==2):
                    val2+=2**(7-i)
            add_dict(dic1, val1)
            add_dict(dic2, val2)
    return dic1, dic2

In [7]:
import cv2
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from scipy.stats import itemfreq
import time
from functools import wraps

df = pd.read_csv(shuffled_labels, header=None)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0


# uncomment below this for local tetra pattern
dic1 = {}
dic2 = {}
dic3 = {}
dic4 = {}

def create_dic():
    for i in range(len(items)):
        dic1[items[i]] = 0
        dic2[items[i]] = 0
        dic3[items[i]] = 0
        dic4[items[i]] = 0

start_time = time.time()      
for row in rows:
    #print(row[1][1])
    create_dic()
    img = cv2.imread(row[1][0])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    new_x = []
    dic1, dic2 = double_lbp(img,dic1, dic2)
    
    x1 = []
    for i in dic1.keys():
        x1.append(dic1[i])
    x1 = np.array(x1)
    x1 = x1/np.sum(x1)
    
    x2 = []
    for i in dic2.keys():
        x2.append(dic2[i])
    x2 = np.array(x2)
    x2 = x2/np.sum(x2)
    '''
    x3 = []
    for i in dic3.keys():
        x3.append(dic3[i])
    x3 = np.array(x3)
    x3 = x3/np.sum(x3)
    
    x4 = []
    for i in dic4.keys():
        x4.append(dic4[i])
    x4 = np.array(x4)
    x4 = x4/np.sum(x4)
    '''
    #new_x = x1.tolist()+x2.tolist()+x3.tolist()+x4.tolist()
    new_x = x1.tolist()+x2.tolist()
    
    X_hist.append(new_x)
    X_addrs.append(row[1][0])
    Y_hist.append(row[1][1])
    if j%100==0:
        elapsed_time = time.time() - start_time
        print(str(j)+" done in " + str(elapsed_time))
        start_time = time.time()
    j = j + 1

Using TensorFlow backend.


0 done in 0.38777756690979004
100 done in 41.37731051445007
200 done in 40.68672561645508
300 done in 43.500110387802124
400 done in 40.399906873703
500 done in 41.305351972579956
600 done in 42.008968114852905
700 done in 40.5128231048584
800 done in 42.72255992889404
900 done in 41.030545473098755
1000 done in 40.361910343170166
1100 done in 43.13430404663086
1200 done in 39.924158573150635
1300 done in 40.12606358528137
1400 done in 43.33520984649658
1500 done in 40.03407835960388
1600 done in 41.32935643196106
1700 done in 43.78794813156128
1800 done in 40.135040044784546


In [8]:
print(X_addrs[0])
print(X_hist[0])
print(Y_hist[0])
#print(X_hist)
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
print(len(X_addrs))
print(len(X_hist[50]))
print(X_hist.shape)
#print((Y_hist))



dataset/catagory_8\D8_180_23.jpg
[0.0, 6.298815822625347e-05, 6.298815822625347e-05, 0.0003149407911312673, 0.0, 0.000818846056941295, 0.002330561854371378, 0.0, 0.0004409171075837742, 0.002771478961955152, 0.003212396069538927, 0.0, 0.0013227513227513227, 0.003401360544217687, 0.008818342151675485, 0.0063618039808516, 0.0, 0.0001889644746787604, 0.0026455026455026454, 0.0037163013353489543, 0.0049130763416477706, 0.0011967750062988158, 0.00012597631645250694, 0.0004409171075837742, 0.0033383723859914337, 0.006739732930209121, 0.006235827664399093, 0.004220206601158982, 0.0004409171075837742, 0.0, 0.0010078105316200555, 0.0025195263290501385, 0.00799949609473419, 0.004976064499874024, 0.0037163013353489543, 0.00012597631645250694, 0.0001889644746787604, 0.0030234315948601664, 0.002708490803728899, 0.0049130763416477706, 0.0011967750062988158, 0.0004409171075837742, 0.0020786092214663643, 0.007747543461829176, 0.004787100025195263, 0.0024565381708238853, 0.0001889644746787604, 0.0018896

In [9]:
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)

print(X_hist.shape, Y_hist.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_hist), np.array(Y_hist), test_size=0.3)
X_train = np.array(X_train)
print(len(X_train))

'''
for i in range(len(X_train)):
    if(len(X_train[i]) != 26):
       print(len(X_train[i]))
'''

print(len(Y_train))

(1880, 118) (1880,)
1316
1316


In [10]:
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
#clf = svm.SVC()
#clf = GaussianNB()
#clf = LogisticRegression()
#clf = MLPClassifier(alpha=1)
#clf = AdaBoostClassifier()
clf = RandomForestClassifier()
#clf = DecisionTreeClassifier()

#X_train = np.array(X_train)
print(type(X_train))

clf.fit(X_train,Y_train)

clf.score(X_test, Y_test)

<class 'numpy.ndarray'>


0.81028368794326244

In [11]:
import pickle
from sklearn.model_selection import cross_val_score
#clf = pickle.load(open('RandomForest_model.sav', 'rb'))
clf = RandomForestClassifier()
clf.fit(X_train,Y_train)
scores = cross_val_score(clf,X_test,Y_test,cv=5)
print((scores))
clf.score(X_test, Y_test)

[ 0.71551724  0.5826087   0.68695652  0.69090909  0.75925926]


0.8014184397163121

In [12]:
import pickle
#for saving model
filename = 'my_double_lbp_model.sav'
pickle.dump(clf, open(filename, 'wb'))
clf = pickle.load(open('my_double_lbp_model.sav', 'rb'))
clf.score(X_test, Y_test)
#scores = cross_val_score(clf, X_test, Y_test, cv=20)
#print((scores))

0.8014184397163121

In [13]:
import numpy as np

X = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
y = np.array([1, 1, 2, 2])

from sklearn.svm import SVC

clf = SVC()
print(type(y))
clf.fit(X, y) 
print(clf.predict([[-0.8, -1]]))

<class 'numpy.ndarray'>
[1]


In [14]:
a = [0]*26
a = np.array(a)
b = a.copy()
b = [0]*14
b = np.array(b)
print(b.shape)
c = a+b
print(c)

(14,)


ValueError: operands could not be broadcast together with shapes (26,) (14,) 

In [ ]:
#lbp-> Local derivative ternary lbp last e  tetra 